This IPython Notebook introduces the use of the `openmc.mgxs` module to calculate multi-group cross sections for an infinite homogeneous medium. In particular, this Notebook introduces the the following features:

* Creation of multi-group cross sections for an **infinite homogeneous medium**
* Use of **tally arithmetic** to manipulate multi-group cross sections

**Note:** This Notebook illustrates the use of Pandas DataFrames to containerize multi-group cross section data. We recommend using Pandas >v0.15.0 or later since OpenMC's Python API leverages the multi-indexing feature included in the most recent releases.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import openmc
import openmc.mgxs as mgxs

%matplotlib inline

We first construct a simple homogeneous infinite medium problem to illustrate use of the `openmc.mgxs` module to generate multi-group cross sections.

## Generate Input Files

First we need to define materials that will be used in the problem. Before defining a material, we must create nuclides that are used in the material.

In [2]:
# Instantiate some Nuclides
h1 = openmc.Nuclide('H-1')
o16 = openmc.Nuclide('O-16')
u235 = openmc.Nuclide('U-235')
u238 = openmc.Nuclide('U-238')
zr90 = openmc.Nuclide('Zr-90')

With the nuclides we defined, we will now create a material for the homogeneous medium.

In [3]:
# Instantiate a Material and register the Nuclides
inf_medium = openmc.Material(name='moderator')
inf_medium.set_density('g/cc', 5.)
inf_medium.add_nuclide(h1,  0.028999667)
inf_medium.add_nuclide(o16, 0.01450188)
inf_medium.add_nuclide(u235, 0.000114142)
inf_medium.add_nuclide(u238, 0.006886019)
inf_medium.add_nuclide(zr90, 0.002116053)

With our material, we can now create a materials file object that can be exported to an actual XML file.

In [4]:
# Instantiate a MaterialsFile, register all Materials, and export to XML
materials_file = openmc.MaterialsFile()
materials_file.default_xs = '71c'
materials_file.add_material(inf_medium)
materials_file.export_to_xml()

Now let's move on to the geometry. This problem will be a simple square cell with reflective boundary conditions to simulate an infinite homogeneous medium. The first step is to create the outer bounding surfaces of the problem.

In [5]:
# Instantiate boundary Planes
min_x = openmc.XPlane(boundary_type='reflective', x0=-0.63)
max_x = openmc.XPlane(boundary_type='reflective', x0=0.63)
min_y = openmc.YPlane(boundary_type='reflective', y0=-0.63)
max_y = openmc.YPlane(boundary_type='reflective', y0=0.63)

With the surfaces defined, we can now create a cell that is defined by intersections of half-spaces created by the surfaces.

In [6]:
# Instantiate a Cell
cell = openmc.Cell(cell_id=1, name='cell')

# Register bounding Surfaces with the Cell
cell.region = +min_x & -max_x & +min_y & -max_y

# Fill the Cell with the Material
cell.fill = inf_medium

OpenMC requires that there is a "root" universe. Let us create a root universe and add our square cell to it.

In [7]:
# Instantiate Universe
root_universe = openmc.Universe(universe_id=0, name='root universe')
root_universe.add_cell(cell)

We now must create a geometry that is assigned a root universe, put the geometry into a geometry file, and export it to XML.

In [8]:
# Create Geometry and set root Universe
openmc_geometry = openmc.Geometry()
openmc_geometry.root_universe = root_universe

# Instantiate a GeometryFile
geometry_file = openmc.GeometryFile()
geometry_file.geometry = openmc_geometry

# Export to "geometry.xml"
geometry_file.export_to_xml()

Next, we must define simulation parameters. In this case, we will use 10 inactive batches and 40 active batches each with 2500 particles.

In [9]:
# OpenMC simulation parameters
batches = 50
inactive = 10
particles = 2500

# Instantiate a SettingsFile
settings_file = openmc.SettingsFile()
settings_file.batches = batches
settings_file.inactive = inactive
settings_file.particles = particles
settings_file.output = {'tallies': True, 'summary': True}
bounds = [-0.63, -0.63, -0.63, 0.63, 0.63, 0.63]
settings_file.set_source_space('fission', bounds)

# Export to "settings.xml"
settings_file.export_to_xml()

Now we are ready to generate multi-group cross sections! First, let's define a 2-group structure using the built-in `EnergyGroups` class.

In [10]:
# Instantiate a 2-group EnergyGroups object
groups = mgxs.EnergyGroups()
groups.group_edges = np.array([0., 0.625e-6, 20.])

We can now use the fine and coarse `EnergyGroups` objects, along with our previously created materials and geometry, to instantiate some `MGXS` objects from the `openmc.mgxs` module. In particular, the following are subclasses of the generic and abstract `MGXS` class:

* `TotalXS`
* `TransportXS`
* `AbsorptionXS`
* `CaptureXS`
* `FissionXS`
* `NuFissionXS`
* `ScatterXS`
* `NuScatterXS`
* `ScatterMatrixXS`
* `NuScatterMatrixXS`
* `Chi`

These classes provide us with an interface to generate the tally inputs as well as perform post-processing of OpenMC's tally data to compute the respective multi-group cross sections. In this case, let's create the multi-group total, absorption and scattering cross sections with our 2-group structure.

In [11]:
# Instantiate a few different sections
total = mgxs.TotalXS(domain=cell, domain_type='cell', groups=groups)
absorption = mgxs.AbsorptionXS(domain=cell, domain_type='cell', groups=groups)
scattering = mgxs.ScatterXS(domain=cell, domain_type='cell', groups=groups)

Each multi-group cross section object stores its tallies in a Python dictionary called `tallies`. We can inspect the tallies in the dictionary for our `Absorption` object as follows. 

In [12]:
absorption.tallies

OrderedDict([('flux', Tally
	ID             =	10000
	Name           =	
	Filters        =	
                		cell	[1]
                		energy	[  0.00000000e+00   6.25000000e-07   2.00000000e+01]
	Nuclides       =	total 
	Scores         =	['flux']
	Estimator      =	tracklength
), ('absorption', Tally
	ID             =	10001
	Name           =	
	Filters        =	
                		cell	[1]
                		energy	[  0.00000000e+00   6.25000000e-07   2.00000000e+01]
	Nuclides       =	total 
	Scores         =	['absorption']
	Estimator      =	tracklength
)])

The `Absorption` object includes tracklength tallies for the 'absorption' and 'flux' scores in the 2-group structure in cell 1. Now that each multi-group cross section object contains the tallies that it needs, we must add these tallies to a `TalliesFile` object to generate the "tallies.xml" input file for OpenMC.

In [13]:
# Instantiate an empty TalliesFile
tallies_file = openmc.TalliesFile()

# Add total tallies to the tallies file
for tally in total.tallies.values():
    tallies_file.add_tally(tally)

# Add absorption tallies to the tallies file
for tally in absorption.tallies.values():
    tallies_file.add_tally(tally)

# Add scattering tallies to the tallies file
for tally in scattering.tallies.values():
    tallies_file.add_tally(tally)
                
# Export to "tallies.xml"
tallies_file.export_to_xml()

Now we a have a complete set of inputs, so we can go ahead and run our simulation.

In [14]:
# Remove old HDF5 (summary, statepoint) files
!rm statepoint.*

# Run OpenMC
executor = openmc.Executor()
executor.run_simulation()


       .d88888b.                             888b     d888  .d8888b.
      d88P" "Y88b                            8888b   d8888 d88P  Y88b
      888     888                            88888b.d88888 888    888
      888     888 88888b.   .d88b.  88888b.  888Y88888P888 888       
      888     888 888 "88b d8P  Y8b 888 "88b 888 Y888P 888 888       
      888     888 888  888 88888888 888  888 888  Y8P  888 888    888
      Y88b. .d88P 888 d88P Y8b.     888  888 888   "   888 Y88b  d88P
       "Y88888P"  88888P"   "Y8888  888  888 888       888  "Y8888P"
__________________888______________________________________________________
                  888
                  888

      Copyright:      2011-2015 Massachusetts Institute of Technology
      License:        http://mit-crpg.github.io/openmc/license.html
      Version:        0.7.0
      Git SHA1:       c4b14a5ef87f004528d35cbf33fef3ed15a386ca
      Date/Time:      2015-11-29 17:50:29
      MPI Processes:  1

 =======================

0

## Tally Data Processing

Our simulation ran successfully and created a statepoint file with all the tally data in it. We begin our analysis here loading the statepoint file and "reading" the results. By default, data from the statepoint file is only read into memory when it is requested. This helps keep the memory use to a minimum even when a statepoint file may be huge.

In [15]:
# Load the last statepoint file
sp = openmc.StatePoint('statepoint.50.h5')

In addition to the statepoint file, our simulation also created a summary file which encapsulates information about the materials and geometry which is necessary for the `openmc.mgxs` module to properly process the tally data. We first create a summary object and link it with the statepoint.

In [16]:
# Load the summary file and link it with the statepoint
su = openmc.Summary('summary.h5')
sp.link_with_summary(su)

The statepoint is now ready to be analyzed by our multi-group cross sections. We simply have to load the tallies from the statepoint into each object as follows and our `MGXS` objects will compute the cross sections for us under-the-hood.

In [17]:
# Load the tallies from the statepoint into each MGXS object
total.load_from_statepoint(sp)
absorption.load_from_statepoint(sp)
scattering.load_from_statepoint(sp)

Voila! Our multi-group cross sections are now ready to rock 'n roll!

## Extracting and Storing MGXS Data

Let's first inspect our total cross section by printing it to the screen.

In [18]:
total.print_xs()

Multi-Group XS
	Reaction Type  =	total
	Domain Type    =	cell
	Domain ID      =	1
	Cross Sections [cm^-1]:
            Group 1 [6.25e-07   - 20.0      MeV]:	6.81e-01 +/- 1.88e-01%
            Group 2 [0.0        - 6.25e-07  MeV]:	1.40e+00 +/- 5.91e-01%





Since the `openmc.mgxs` module uses tally arithmetic under-the-hood, the cross section is stored as a "derived" tally. This means that it can be queried and manipulated using all of the same method supported for the `Tally` class in the OpenMC Python API. For example, we can construct a Pandas DataFrame of the multi-group cross section data.

In [19]:
df = scattering.get_pandas_dataframe()
df.head(10)

/usr/local/lib/python2.7/dist-packages/openmc-0.7.0-py2.7.egg/openmc/mgxs/mgxs.py:1254: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


,cell,group in,nuclide,mean,std. dev.
1,1,1,total,0.668323,0.001264
0,1,2,total,1.293258,0.007624


Each multi-group cross section object can be easily exported to a variety of file formats, including CSV, Excel, and LaTeX for storage or data processing.

In [20]:
absorption.export_xs_data(filename='absorption-xs', format='excel')

The following code snippet shows how to export all of three cross sections to the same HDF5 binary data store.

In [21]:
total.build_hdf5_store(filename='mgxs', append=True)
absorption.build_hdf5_store(filename='mgxs', append=True)
scattering.build_hdf5_store(filename='mgxs', append=True)

## Comparing MGXS with Tally Arithmetic

Finally, we illustrate how one can leverage OpenMC's tally arithmetic data processing feature with `MGXS` objects. The `openmc.mgxs` module uses tally arithmetic to compute multi-group cross sections with automated uncertainty propagation. Each `MGXS` object includes an `xs_tally` attribute which is a "derived" tally based on the tallies needed to compute the cross section type of interest. These derived tallies can be used in subsequent tally arithmetic operations. For example, we can use tally artithmetic to confirm that the `TotalXS` is equal to the sum of the `AbsorptionXS` and `ScatterXS` objects.

In [22]:
# Use tally arithmetic to compute the difference between the total, absorption and scattering
difference = total.xs_tally - absorption.xs_tally - scattering.xs_tally

# The difference is a derived tally which can generate Pandas DataFrames for inspection
difference.get_pandas_dataframe()

,cell,energy [MeV],nuclide,score,mean,std. dev.
0,1,(0.0e+00 - 6.3e-07),total,(((total / flux) - (absorption / flux)) - (sca...,4.884981e-15,0.011274
1,1,(6.3e-07 - 2.0e+01),total,(((total / flux) - (absorption / flux)) - (sca...,1.221245e-15,0.001802


Similarly, we can use tally arithmetic to compute the ratio of `AbsorptionXS` and `ScatterXS` to the `TotalXS`.

In [23]:
# Use tally arithmetic to compute the absorption-to-total MGXS ratio
absorption_to_total = absorption.xs_tally / total.xs_tally

# The absorption-to-total ratio is a derived tally which can generate Pandas DataFrames for inspection
absorption_to_total.get_pandas_dataframe()

,cell,energy [MeV],nuclide,score,mean,std. dev.
0,1,(0.0e+00 - 6.3e-07),total,((absorption / flux) / (total / flux)),0.076219,0.000651
1,1,(6.3e-07 - 2.0e+01),total,((absorption / flux) / (total / flux)),0.019319,0.000086


In [24]:
# Use tally arithmetic to compute the scattering-to-total MGXS ratio
scattering_to_total = scattering.xs_tally / total.xs_tally

# The scattering-to-total ratio is a derived tally which can generate Pandas DataFrames for inspection
scattering_to_total.get_pandas_dataframe()

,cell,energy [MeV],nuclide,score,mean,std. dev.
0,1,(0.0e+00 - 6.3e-07),total,((scatter / flux) / (total / flux)),0.923781,0.007714
1,1,(6.3e-07 - 2.0e+01),total,((scatter / flux) / (total / flux)),0.980681,0.002617


In [25]:
# Use tally arithmetic to ensure that the absorption- and scattering-to-total MGXS ratios sum to unity
sum_ratio = absorption_to_total + scattering_to_total

# The scattering-to-total ratio is a derived tally which can generate Pandas DataFrames for inspection
sum_ratio.get_pandas_dataframe()

,cell,energy [MeV],nuclide,score,mean,std. dev.
0,1,(0.0e+00 - 6.3e-07),total,(((absorption / flux) / (total / flux)) + ((sc...,1,0.007741
1,1,(6.3e-07 - 2.0e+01),total,(((absorption / flux) / (total / flux)) + ((sc...,1,0.002619
